# PAC-Bayes

* [A Primer on PAC-Bayesian Learning](https://arxiv.org/abs/1901.05353)

これまでの統計的学習理論([MATH_statistical_learning_theory.ipynb](MATH_statistical_learning_theory.ipynb))では予測器を点推定することを考えてきました。
PACベイズでは分布推定することを考えます。

例えば教師あり学習の回帰タスクで、

* 入力: $X\in \mathbb{R}^d$
* 出力: $Y\in \mathbb{R}$
* ノイズ: $W\in \mathbb{R}$

について、

$$
Y=f(X)+W
$$

の出力変数を回帰することを考えます。このとき、一般のベイズの枠組みでは

$$
\operatorname{posterior}(f \mid X, Y) \propto \operatorname{likelihood}(X, Y \mid f) \times \operatorname{prior}(f)
$$

の形式で推論を行います。もしくはより一般な枠組みとして、パラメータ$\lambda$によって尤度の影響をコントロールして、

$$
\operatorname{target}(f \mid X, Y) \propto \operatorname{likelihood}(X, Y \mid f)^\lambda \times \operatorname{prior}(f)
$$

の形式で推論します（これもproperな事後分布になります。つまり、$\int$や$\sum$が無限に飛んだりしません。）


## ベイズ学習の枠組み

* データ：$\mathcal{D}_n=(X_i, Y_i)_{i=1}^n$
    * $(X, Y) \in \mathbb{R}^d \times \mathbb{R}$
* 学習アルゴリズム：データのサンプルを予測器にマップするもの
    * $\cup_{j=1}^{\infty}\left(\mathbb{R}^d \times \mathbb{R}\right)^j \rightarrow \mathcal{F}$
* 損失関数：$\ell: \mathbb{R} \times \mathbb{R} \rightarrow \mathbb{R}_{+}$
    * 予測器$\hat{\phi}$の汎化能力（リスク）：$R: \widehat{\phi} \mapsto \mathbb{E}[\ell(\widehat{\phi}(X), Y)]$
    * 経験予測誤差：$r_n: \widehat{\phi} \mapsto \frac{1}{n} \sum_{i=1}^n \ell\left(\widehat{\phi}\left(X_i\right), Y_i\right)$
        * $\mathbb{E}\left[r_n(\widehat{\phi})\right]=R(\widehat{\phi})$が成立

### ベイズ学習

$\widehat{\phi}$がベイズ予測器であるときを考えましょう。
このとき、事前分布$\pi_0$として、ベイズ予測器は予測関数の集合（$\mathcal{F}=\left\{f: \mathbb{R}^d \rightarrow \mathbb{R}, f\right.$ mesurable $\}$）もしくはその部分集合について考えます。事前集合はその集合を台とします。

例えば
$$
\pi\left(\widehat{\phi} \mid \mathcal{D}_n\right) \propto \mathcal{L}\left(\mathcal{D}_n \mid \widehat{\phi}\right) \times \pi_0(\widehat{\phi})
$$
のようにして学習します。

この事後分布から、様々な派生が得られます。

* 平均値：$\widehat{\phi}^{\text {mean }}=\mathbb{E}_\pi \phi=\int_{\mathcal{F}_0} \phi \pi(\mathrm{d} \phi)$
* 中央値：$\widehat{\phi}^{\text {median }}=\operatorname{median}(\pi)$
* MAP推定：$\widehat{\phi}^{\mathrm{MAP}} \in \underset{\phi \in \mathcal{F}_0}{\arg \max } \pi(\phi)$
* 単一の実現値：$\widehat{\phi}^{\text {draw }} \sim \pi$

### 一般化ベイズ学習

最初に紹介したように、適当なパラメータによって尤度の影響を和らげることができます。

$$
\pi_\lambda\left(\widehat{\phi} \mid \mathcal{D}_n\right) \propto \mathcal{L}\left(\mathcal{D}_n \mid \widehat{\phi}\right)^\lambda \times \pi_0(\widehat{\phi})
$$

これは$\lambda=1$よりも良い性質を持ちます（らしいです）。実際、$\lambda\neq 1$の場合は最良の予測器に収束することが保証できますが、$\lambda=1$だとダメなケースがあるみたいですね（7ページ参照）。

この尤度の影響をコントロールするフレームワークでもいいですが、尤度の部分をまるごと損失関数として扱うこともできます。このとき、次の状況を考えます（一般化事後分布）。

$$
\pi_\lambda\left(\widehat{\phi} \mid \mathcal{D}_n\right) \propto \ell_{\lambda, n}(\widehat{\phi}) \times \pi_0(\widehat{\phi})
$$

例えば次のようなGibbs分布による事後分布がよく使われます。
$$
\pi_\lambda\left(\widehat{\phi} \mid \mathcal{D}_n\right) \propto \exp \left[-\lambda r_n(\widehat{\phi})\right] \times \pi_0(\widehat{\phi})
$$

$\lambda=0$なら事前分布の情報しか使いません。一方で、$\lambda\to \infty$なら経験リスクを最小化する、いわゆるERM (Empirical Risk Minimization)になります。


### Gibbs分布による事後分布について

* $(A, \mathcal{A})$：可測空間
* $\mu, \nu$：$(A, \mathcal{A})$上の確率測度。$\mu$は$\nu$について絶対連続であるとする。つまり、

$$
\mathcal{M}_\nu(A, \mathcal{A})=\{\mu: \mu \ll \nu\}
$$

**正則化された回帰との関係**

２つの確率測度$\mu, \nu$のKLダイバージェンスを

$$
\mathcal{K}(\mu, \nu)= \begin{cases}\int_{\mathcal{F}_0} \log \left(\frac{\mathrm{d} \mu}{\mathrm{d} \nu}\right) \mathrm{d} \mu & \text { when } \mu \ll \nu \\ +\infty & \text { otherwise }\end{cases}
$$

とします。

このとき、

$$
\underset{\mu \in \mathcal{M}_{\pi_0}(A, \mathcal{A})}{\arg \inf }\left\{\int_{\mathcal{F}_0} r_n(\phi) \mu(\mathrm{d} \phi)+\frac{\mathcal{K}\left(\mu, \pi_0\right)}{\lambda}\right\}
$$

なる最小化問題を考えます。
つまり、経験リスクを最小化しながら、$\pi_0$からあまり離れないような$\phi$についての確率測度（分布）を求めることになります。
これは二乗誤差に正則化をつけた場合とほぼ同じです（実際、KKT条件を考えれば簡単に導出できます）。

**Exponential Weighted Aggregateとの関係**

Gibbs分布による事後分布について、その期待値を変形してみると、

$$
\begin{aligned}
\widehat{\phi}^{\text {mean }}:=\mathbb{E}_{\pi_\lambda} \phi & =\int_{\mathcal{F}_0} \phi \pi_\lambda(\mathrm{d} \phi) \\
& =\int_{\mathcal{F}_0} \phi \exp \left[-\lambda r_n(\phi)\right] \pi_0(\mathrm{~d} \phi) \\
& =\sum_{i=1}^{\# \mathcal{F}_0} \frac{\exp \left[-\lambda r_n\left(\phi_i\right)\right] \pi\left(\phi_i\right)}{\underbrace{\sum_{j=1}^{\# \mathcal{F}_0} \exp \left[-\lambda r_n\left(\phi_j\right)\right] \pi\left(\phi_j\right)}_{=: \omega_{\lambda, i}}} \phi_i=\sum_{i=1}^{\# \mathcal{F}_0} \omega_{\lambda, i} \phi_i,
\end{aligned}
$$

であり、これも統計の文脈でよく使われるアルゴリズムです。

## PAC Bayesの枠組み

PAC Bayesではベイズ学習で得られた分布$\mu$についてのPACバウンドを考えます。例えば次の不等式が便利です。

---

**McAllesterのバウンド**

任意の$\mu\in \mathcal{M}_{\pi_0}(A, \mathcal{A})$と$\epsilon > 0$について、

$$
\mathbb{P}\left[\int R(\widehat{\phi}) \mathrm{d} \mu(\widehat{\phi}) \leq \int r_n(\widehat{\phi}) \mathrm{d} \mu(\widehat{\phi})+\sqrt{\frac{\mathcal{K}\left(\mu, \pi_0\right)+\log \frac{2 \sqrt{n}}{\epsilon}}{2 n}}\right] \geq 1-\epsilon .
$$

が成り立つ。
これはKLダイバージェンスて予測器の集合$\mathcal{F}_0$の複雑度に対応することに注意しよう。

---

このバウンドは便利ですが、実際に使う予測器(例えば$\widehat{\phi}^{\text {mean }}$)についてのバウンドは何も言っていないことに注意しましょう。
これを求めるためには次のCatoniのバウンドが便利です。


### Catoniのバウンド

Catoniのバウンドは次の２つのテクニックから成ります：

1. 適当な$\widehat{\phi} \in \mathcal{F}_0$に対して、リスク$R(\widehat{\phi})$と経験リスク$r_n(\widehat{\phi})$の差をバウンドします。Azuma-Hoeffdingなどでやります。
2. 任意の$\hat{\phi} \in \mathcal{F}_0$に対してのバウンドを求めます。このときに次の補題が便利です。

---

* $(A, \mathcal{A})$：可測空間

$(A, \mathcal{A})$上の任意の分布$\nu$と$\int(\exp \circ h) \mathrm{d} \nu<\infty$を満たすような可測関数$h: A \to \mathbb{R}$に対して

$$
\log \int(\exp \circ h) \mathrm{d} \nu=\sup _{\mu \in \mathcal{M}_\nu(A, \mathcal{A})}\left\{\int h \mathrm{~d} \mu-\mathcal{K}(\mu, \nu)\right\}
$$

である。（ここで$\infty-\infty=-\infty$とする）。

また、$h$に上界がある場合は、右辺は次で与えられるGibbs分布$g$が解析解になる。

$$
\frac{\mathrm{d} g}{\mathrm{~d} \nu}(a)=\frac{\exp \circ h(a)}{\int(\exp \circ h) \mathrm{d} \nu}, \quad a \in A
$$

**証明**

KLダイバージェンスは非負なので、
$\mu \mapsto-\mathcal{K}(\mu, g)$は$\mu=g$のときに最大値を取る。
すると、

$$
\begin{aligned}
-\mathcal{K}(\mu, g) & =-\int \log \left(\frac{\mathrm{d} \mu}{\mathrm{d} \nu} \frac{\mathrm{d} \nu}{\mathrm{d} g}\right) \mathrm{d} \mu \\
& =-\int \log \left(\frac{\mathrm{d} \mu}{\mathrm{d} \nu}\right) \mathrm{d} \mu+\int \log \left(\frac{\mathrm{d} g}{\mathrm{~d} \nu}\right) \mathrm{d} \mu \\
& =-\mathcal{K}(\mu, \nu)+\int h \mathrm{~d} \mu-\log \int(\exp \circ h) \mathrm{d} \nu
\end{aligned}
$$

であるから、全ての$\mu$についてsupremumを取れば、左辺は$0$であるから、変形して

$$
\log \int(\exp \circ h) \mathrm{d} \nu=\sup _{\mu \in \mathcal{M}_\nu(A, \mathcal{A})}\left\{\int h \mathrm{~d} \mu-\mathcal{K}(\mu, \nu)\right\}
$$

---

この補題について、$\nu=\pi_0$とし、$h=-\lambda r_n$とすれば、

$$
-\frac{1}{\lambda} \log \int \exp \left[-\lambda r_n(\phi)\right] \pi_0(\mathrm{~d} \phi)=\inf _{\mu \in \mathcal{M}_{\pi_0}(A, \mathcal{A})}\left\{\int r_n(\phi) \mu(\mathrm{d} \phi)+\frac{\mathcal{K}\left(\mu, \pi_0\right)}{\lambda}\right\}
$$

を得ます。ここで、最初の方にやったGibbs分布の形の事後分布

$$
\pi_\lambda\left(\widehat{\phi} \mid \mathcal{D}_n\right) \propto \exp \left[-\lambda r_n(\widehat{\phi})\right] \times \pi_0(\widehat{\phi})
$$

を考えてみましょう。これとHoeffdingなどを使えば、

$$
\mathbb{P}\left[\int R(\phi) \pi_\lambda(\mathrm{d} \phi) \leq \inf _{\mu \in \mathcal{M}_{\pi_0}(A, \mathcal{A})}\left\{\int R(\phi) \mu(\mathrm{d} \phi)+\frac{\lambda B}{n}+\frac{2}{\lambda}\left(\mathcal{K}\left(\mu, \pi_0\right)+\log \frac{2}{\epsilon}\right)\right\}\right] 
\geq 1-\epsilon
$$

が導出できます（TODO: 導出。）

この右辺に注目すれば、最適な$\lambda$もここから求めることができます。
また、Jensenの不等式を使えば

$$
\mathbb{P}\left[R\left(\widehat{\phi}^{\text {mean }}\right) \leq \inf _{\mu \in \mathcal{M}_{\pi_0}(A, \mathcal{A})}\left\{\int R(\phi) \mu(\mathrm{d} \phi)+\frac{\lambda B}{n}+\frac{2}{\lambda}\left(\mathcal{K}\left(\mu, \pi_0\right)+\log \frac{2}{\epsilon}\right)\right\}\right]
\leq 1 - \epsilon
$$

も求まります。